In [2]:
'''
    @author:
        - Leonardo Lo Schiavo
    @affiliation:
        - IMDEA Networks institute
'''
import torch, os
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd


In [5]:
'''
    @author:
        - Leonardo Lo Schiavo
    @affiliation:
        - IMDEA Networks institute
'''
class Predictor(nn.Module):

    def __init__(self, input_size, output_size, hidden_size, nonlin=F.relu, norm_in=False):

        super(Predictor, self).__init__()

        if norm_in:
            self.in_fn = nn.BatchNorm1d(input_size, affine=False)
        else:
            self.in_fn = lambda x: x

        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.nonlin = nonlin


    def forward(self, X):

        inp = self.in_fn(X)
        h1 = self.nonlin(self.fc1(inp))
        h2 = self.nonlin(self.fc2(h1))
        out = self.fc3(h2)
        return out

In [ ]:
'''
    @author:
        - Leonardo Lo Schiavo
    @affiliation:
        - IMDEA Networks institute
'''
class LPUModels:

    def __init__(self):
        # Dataset parameters
        self.max_snr = 30.0
        self.max_mcs = 27
        self.max_total_bits = 295680
        self.max_prbs = 250

        # LPU models parameters
        self.input_size = 3
        self.output_size = 1
        self.hidden_size = 128

        # Import max and min values for the inputs (only GPU power model)
        self.max_gpu_power = np.load('/home/jovyan/data/max_power_gpu.npy')
        self.min_gpu_power = np.load('/home/jovyan/data/min_power_gpu.npy')

        # Load the model weights and set the model to inference mode
        self.predictor_time_cpu = Predictor(self.input_size, self.output_size, self.hidden_size)
        self.predictor_time_cpu.load_state_dict(torch.load('/home/jovyan/data/predictor_time_cpu.pyt', map_location="cpu"))
        self.predictor_time_cpu.eval()
        self.predictor_time_gpu = Predictor(self.input_size, self.output_size, self.hidden_size)
        self.predictor_time_gpu.load_state_dict(torch.load('/home/jovyan/data/predictor_time_gpu.pyt', map_location="cpu"))
        self.predictor_time_gpu.eval()
        self.predictor_power_cpu = Predictor(self.input_size, self.output_size, self.hidden_size)
        self.predictor_power_cpu.load_state_dict(torch.load('/home/jovyan/data/predictor_power_cpu.pyt', map_location="cpu"))
        self.predictor_power_cpu.eval()
        self.predictor_power_gpu = Predictor(self.input_size, self.output_size, self.hidden_size)
        self.predictor_power_gpu.load_state_dict(torch.load('/home/jovyan/data/predictor_power_gpu.pyt', map_location="cpu"))
        self.predictor_power_gpu.eval()


    def estimate_service_time(self, snr, mcs, prbs, total_bits):
        # Normalize the inputs and format them for PyTorch
        power_inputs = []
        power_inputs.append(snr / self.max_snr)
        power_inputs.append(mcs / self.max_mcs)
        power_inputs.append(prbs / self.max_prbs)
        power_inputs = torch.Tensor(power_inputs)

        time_inputs = []
        time_inputs.append(snr / self.max_snr)
        time_inputs.append(mcs / self.max_mcs)
        time_inputs.append(total_bits / self.max_total_bits)
        time_inputs = torch.Tensor(time_inputs)
        # Run the model in inference mode
        power_cpu = self.predictor_power_cpu(power_inputs).detach().numpy()
        time_cpu = float(self.predictor_time_cpu(time_inputs).detach().numpy())
        energy_cpu = float(power_cpu * time_cpu)

        power_gpu = self.predictor_power_gpu(power_inputs).detach().numpy() * (self.max_gpu_power - self.min_gpu_power) + self.min_gpu_power
        time_gpu = float(self.predictor_time_gpu(time_inputs).detach().numpy())
        energy_gpu = float(power_gpu * time_gpu)

        return time_cpu, energy_cpu, time_gpu, energy_gpu

In [ ]:
'''
    @author:
        - Leonardo Lo Schiavo
    @affiliation:
        - IMDEA Networks institute
'''
input_file = "/home/jovyan/data/trace_40BS.csv"

# Load Trace
trace_df = pd.read_csv(input_file, sep=",", header=0)
trace_df.value_counts()

# Create an instance of LPU Models
lpu_models = LPUModels()

# Run inference for each request within the trace
results_df = pd.DataFrame(columns = ['t_cpu', 'e_cpu', 't_gpu', 'e_gpu'])
for index, row in trace_df.iterrows():
    t_cpu, e_cpu, t_gpu, e_gpu = lpu_models.estimate_service_time(row['SNR'], row['MCS'], row['PRBs'], row['TBS'])
    tmp_df = pd.DataFrame([[t_cpu, e_cpu, t_gpu, e_gpu]], columns=['t_cpu', 'e_cpu', 't_gpu', 'e_gpu'])
    results_df = pd.concat([results_df, tmp_df], ignore_index = True)

print(results_df.head(1))

/tmp/ipykernel_92/1786070879.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  time_cpu = float(self.predictor_time_cpu(time_inputs).detach().numpy())
/tmp/ipykernel_92/1786070879.py:56: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  energy_cpu = float(power_cpu * time_cpu)
/tmp/ipykernel_92/1786070879.py:59: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  time_gpu = float(self.predictor_time_gpu(time_inputs).detach().numpy())
/tmp/ipykernel_92/1786070879.py:60: De